In [33]:
## personal imports
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime

from tradealgorithm.tradealgorithm import TradeAlgorithm
from tradealgorithm.tradealgorithmfactory import TradeAlgorithmFactory
from pricer.pricer_factory import PricerFactory
from pricer.pricer import Pricer
from ranker.ranker_factory import RankerFactory
from ranker.ranker import Ranker
from classifier.classifier_factory import ClassifierFactory
from classifier.classifier import Classifier
from datetime import datetime

## risk and return class imports
from returns.products import Products as returns_products_class

## standard imports
from statistics import variance
import math
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [34]:
start = datetime(2001,1,1)
end  = datetime(2023,1,1)
current_date = datetime(2023,1,1)

In [35]:
market = Market()

In [36]:
trade_algo = TradeAlgorithmFactory.build(TradeAlgorithm.RRR_BETA)
pricer_class = Pricer.DAILY_STOCK_ROLLING
ranker_class = Ranker.NONE
classifier_class = Classifier.NONE
current = False
trade_algo.initialize(pricer_class,ranker_class,classifier_class,start,end,current_date)
trade_algo.initialize_bench_and_yields()
trade_algo.initialize_classes()

In [37]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [38]:
market.connect()
tyields = returns_products_class.tyields(market.retrieve("tyields10"),10)
bench_returns = returns_products_class.spy_bench(market.retrieve("spy"))
market.disconnect()

In [39]:
analysis = []
trade_algo.db.connect()
parameters = trade_algo.pull_iterations().to_dict("records")
for parameter in tqdm(parameters):
    try:
        trade = trade_algo.db.query("trades",{"iteration":parameter["iteration"]})
        if trade.index.size > 0:
            positions = int(trade_algo.positions) if trade_algo.pricer_class.asset_class.value == "stocks" else 1
            pricer_timeframe =  trade_algo.pricer_class.time_horizon_class.naming_convention
            trade = trade[trade["position"] <= positions]
            ledger = trade.sort_values(["year",pricer_timeframe]).copy()
            anas = trade_algo.analysis
            if ledger.index.size > 0:
                cumulative = anas.trade_analysis(ledger,positions,tyields,bench_returns)
                for key in parameter.keys():
                    cumulative[key] = parameter[key]
                cumulative["trade_algo"] = trade_algo.name
                for key in parameter.keys():
                    cumulative[key] = parameter[key]
                analysis.append(cumulative[list(parameter.keys()) + ["trade_algo","pv","beta","rrr","sharpe"]].iloc[-1])
    except Exception as e:
        print(str(e))
        continue
trade_algo.db.disconnect()
a = pd.DataFrame([x.to_dict() for x in analysis])

0it [00:00, ?it/s]


In [40]:
a.sort_values("pv",ascending=False).head(20)

KeyError: 'pv'

In [ ]:
a.sort_values("pv",ascending=False).groupby("trade_algo").nth(0).reset_index()

In [ ]:
top = a.sort_values("pv",ascending=False).iloc[0]
db = ADatabase(top["trade_algo"])
db.connect()
analysis = db.query("trades",{"iteration":int(top["iteration"])})
db.disconnect()

In [ ]:
anas = trade_algo.analysis
naming = "date"
positions = int(int(analysis["position"].max())/5)
ledger = analysis[["year",naming,"ticker",f"{naming}ly_delta",f"{naming}ly_delta_sign","actual_returns","position"]]
ledger = ledger[ledger["year"]>=start.year]
ledger = ledger.merge(sp500[["ticker","GICS Sector","Security"]],on="ticker",how="left")

In [ ]:
ledger

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
plt.scatter(ledger[f"{naming}ly_delta"] * 100, (ledger["actual_returns"]-1)*100,s=10)

In [ ]:
ledger.sort_values(["year",naming,"position"],ascending=True).head(20)

In [ ]:
ledger[ledger["position"]<20].sort_values("actual_returns",ascending=False).head(50)

In [ ]:
ledger["actual_returns"].mean()

In [ ]:
ledger[ledger["actual_returns"]>1].index.size / ledger.index.size

In [ ]:
portfolio = ledger.pivot_table(index=["year",naming],columns="position",values="actual_returns").fillna(1).reset_index()

In [ ]:
portfolio

In [ ]:
positions = 10

In [ ]:
cumulative = anas.iteration_analysis(portfolio,positions,bench_returns)

In [ ]:
cumulative.tail(10)

In [ ]:
cash_analysis = cumulative.copy()
cash_analysis["previous_day"] = cash_analysis["daily_returns"].shift(1)
cash_analysis["base"] = [min(row[1]["previous_day"],1) for row in cash_analysis.iterrows()]
cash_analysis["real_return"] = cash_analysis["base"] * cash_analysis["daily_returns"]
cash_analysis["cash"] = [max(0,row[1]["real_return"]-1) * 100 for row in cash_analysis.iterrows()]
cash_analysis["running_cash"] = cash_analysis["cash"].cumsum()

In [ ]:
cash_analysis["cash"].sum()

In [ ]:
cash_analysis[["daily_returns","base","real_return","cash","running_cash"]]

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
plt.plot(cash_analysis["date"],cash_analysis["running_cash"])

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
plt.plot(cumulative["date"],cumulative["pv"])
plt.plot(cumulative["date"],cumulative["bench"])
plt.legend(["pv","bench"])

In [ ]:
cumulative.tail(20)

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
position_columns = [x for x in cumulative.columns if x not in ["year","week","quarter","date_string","date","pv","adjclose","bench"]]
for i in position_columns:
    position = cumulative[i]
    plt.plot(cumulative["date"],position)
plt.show()


In [ ]:
ledger.groupby(["GICS Sector"]).mean().sort_values("actual_returns",ascending=False)

In [ ]:
ledger.groupby(["year",naming,"ticker","Security"]).prod().sort_values("actual_returns",ascending=False).head(30)

In [ ]:
industry_analysis = ledger.pivot_table(index=["year",naming],columns="GICS Sector",values="actual_returns").fillna(1).reset_index()
industry_analysis[[x for x in industry_analysis.columns if x != "sell_date"]].cumprod()